# Simplicial Complexes

A simplicial complex is a geometric figure obtained by gluing simplices of various dimensions along the faces. A simplex of dimesion $n$ is the covex hull of the set of standard basis bactors $\{e_1,\ldots, e_{n+1}\}$ in $\mathbb{R}^{n+1}$. Therefore a $0$-simplex is a point, a $1$-simplex is a line segment, a $2$-simplex is a triangle, a $3$-simplex is a tetrahedron and so on. 

In a simplcial complex every simplex is uniquely expressed by the collection of its vertices. For instance a vertex set $\sigma =\{0,\ldots,m\}$ forms the $m+1$ vertices of a $m$-simplex $\sigma$. The simplex $\sigma$ has $m+1$ faces denoted by $d_i(\sigma), 0\leq i \leq m$, which are $m$-simplices, associated with them. Here $$d_i(\sigma) = \{0,\ldots,\hat{i}, \ldots,m\}$$ obtained by omitting the $i$-th vertex. 

An abstract simplicial complex is a pair $X = (V,\Sigma)$ where $V$ is a finite set and $\Sigma \subset \mathcal{P}(V)$ is a collection of subsets of $V$ with the following property: If $\sigma \in V$ and $\tau \subset \Sigma$, then $\tau \in \Sigma$.

Given such a pair $X = (V,\Sigma)$ we form a simplicial complex as follows. The sets of vertices are in bijection with the set $V$. An ordering on $V$ gives and ordering on the vertex set. If $V = \{a_0,\ldots,a_n\}$, a subset $\sigma = \{a_{k_0} \ldots a_{k_n}\}$ (preserving the ordering of $V$) denotes the corresponding $n$-simplex. For example we may model a circle using the abstract simplcial complex with $V = \{1,2,3\}$ and $\Sigma = \{\{1,2\},\{2,3\}, \{1,3\},\{1\}, \{2\},\{3\}\}$.

Given an abstract simplicial complex $(V,\Sigma)$, let $\Sigma_k \subset \Sigma$ be the subset of $k$-simplices. There are faces maps $$d_i: \Sigma_k \to \Sigma_{k-1}, \, 0 \leq i \leq k$$ $$d_i(\{a_{l_0},\ldots,a_{l_n} \}) = \{a_{l_0},\ldots,\widehat{a_{l_i}}, \ldots, a_{l_n}\}$$


# Simplicial Homology

Let $C_k = \mathbb{Z}(\Sigma_k)$ be the free abelian group generated by the set of $k$-simplices. There is a boundary maps of abelian groups $$\partial_k: C_k \to C_{k-1}$$ $$\partial_k = \sum_{i=0}^{n}(-1)^id_i$$ The boundary maps have the property that $\partial_{k-1}\circ \partial_k = 0$.

This forms a chain complex of abelian groups $C_{\bullet} = \{C_k\}, \, (k\geq 0)$ connected by boundary maps $\partial_k$. The $k$-th homology group of the simplicial complex is the $k$-homology of the chain complex, $$H_k^{\Delta}(X) = \mbox{Ker}(\partial_k)/\mbox{Im}(\partial_{k+1}).$$ Given an abelian group $A$, the simplicial homology groups with coefficients in $A$ is the homology of the chain complex $C_{\bullet}\otimes A$: $$H_k^{\Delta}(X;A) = H_k(C_{\bullet}\otimes A)$$

The Betti numbers are defined as $$\beta(X)_k = \dim_FH_k^{\Delta}(X;F)$$ wher $F$ is a field. The $k$-Betti is the number of $k$-dimensional holes in the topological realization of $X$. 

# Python implementation

An abstract simplicial complex can be encoded as a set of sets. In python we implement this as a list of lists.

For example one can implement a simplcial complex structure of a circle as follows:

$X$ = ["x","y","z", "xy", "yz", "xz"]


To every simplex in $X$ we can associate a simplex "tree" consisting of all its faces in lower dimensions. We use a list of lists to use a write a tree. The simplex tree associated with the simplex "xy" is ["xy", ["x",[]], ["y",[]]]. The simplex trees of all the simplices can be combined to form a $X.\mbox{dim}+1$-partitite graph (also written as a list of lists), called the facetree, encoding the face maps in its entirety. The facetree of $X$ is ["xy",["x",[]],"yz",["y",[],"z",[]], "xz",["x",[]],["z",[]]].



   
## Computing Betti numbers

Since the groups $C_k(X)$ are equipped with bases $\Sigma_k$, the operators $\partial_k$ can be expressed as matrices $D(k)$ whose rows are indexed by $\Sigma_{k-1}$ and columns are indexed by $\Sigma_{k}$. For $\sigma \in \Sigma_{k}$ and $\tau \in \Sigma_{k-1}$ the entry $D(k)_{\sigma\tau} = 0$ if $\tau \not\subset \sigma$ and $=(-1)^i$ if $\tau \subset \sigma$ and $\tau$ is obtained from $\sigma$ by removing the $i$-th element of the subset $\sigma$.


$$\beta(X)_k = \dim_F\left( \frac{\mbox{Ker}\,D(k)}{\mbox{Im}\,D(k+1)} \right) = \mbox{len}(\Sigma_k) - \mbox{rank}(D(k)) - \mbox{rank}(D(k+1))$$






In [1]:
import numpy as np

In [2]:
def flatten(A):
	if A == []: return A
	if type(A[0]) == list:
		return flatten(A[0]) + flatten(A[1:])
	else: return [A[0]] + flatten(A[1:])


In [3]:

class scomplex():
	def __init__(self, simplices = []):
		self.simplices = list(set(flatten([self.simplexfaces(simplex) for simplex in simplices])))
		self.vertices = [simplex for simplex in self.simplices if len(simplex)-1 == 0]
		self.dim = max(len(x)-1 for x in self.simplices)
		self.topdimsimplices = [simplex for simplex in self.simplices if len(simplex)-1 == self.dim]
		self.topsimplices = self.topdimsimplices  
		self.facetree = self.facetree()

	def Sigma(self,k):
		return [simplex for simplex in self.simplices if len(simplex)-1 == k]
	
	def d(self,simplex,i):
		if i in range(len(simplex)):
			return simplex[0:i] + simplex[i+1:len(simplex)]
		else:pass

	def simplextree(self, simplex):
		simplextree = []
		if len(simplex) == 1: return [simplex,[]]
		else:
			simplextree = [simplex]
			for i in range(len(simplex)):
				simplextree.append(self.simplextree(self.d(simplex,i)))
		return simplextree
	

	def simplexfaces(self,simplex):
		return flatten(self.simplextree(simplex))


	def facetree(self):
		return [self.simplextree(simplex) for simplex in self.topsimplices]	

	def D(self,k):
		if k in range(self.dim +1):
			if k == 0: return np.zeros(len(self.Sigma(0)))
			else:
				n = len(self.Sigma(k))
				m = len(self.Sigma(k-1))
				A = np.zeros((m,n))
				for i in range(m):
					for j in range(n):
						for l in range(k+1):
							if self.Sigma(k-1)[i] == self.d(self.Sigma(k)[j],l):
								A[i,j] = (-1)**l
							else: pass
				return A
		elif k == self.dim+1: return np.zeros((1,1))	
		else: pass


	def betti(self):
		return [len(self.Sigma(k)) - np.linalg.matrix_rank(self.D(k)) - np.linalg.matrix_rank(self.D(k+1)) for k in range(self.dim +1)]


In the implementation it is not necessary to enter all the simplices. The scomplex class will automatically include all the lower dimensional faces of a simplex. The class scomplex has attributes simplices (lists down the simplices of all dimensions), Sigma($i$) = list of $i$ dimensional simplcies, and betti = list of betti numbers from dimesion $0$ to dimension of the complex.

The $0$-th Betti number is the number of connected components and the $i$-th Betti number (for $i>0$) is the number of $i$-dimensional holes in the topological realization of the simplicial complex.


In [4]:
Z = scomplex(["xy","yz","xz","zw","xyz","yzw","xzw","xyw", "xa","abcd"])

print("simplices =", Z.simplices, "\n")
print("Sigma_0 = ", Z.Sigma(0),"\n")
print("Sigma_1 = ", Z.Sigma(1),"\n")
print("Sigma_2 = ", Z.Sigma(2),"\n")


print("Betti numbers =",Z.betti())

simplices = ['z', 'd', 'xw', 'bc', 'ab', 'x', 'xzw', 'a', 'w', 'acd', 'xz', 'bd', 'yzw', 'cd', 'y', 'yz', 'zw', 'xa', 'abc', 'xyw', 'ad', 'yw', 'ac', 'abd', 'xy', 'abcd', 'xyz', 'bcd', 'c', 'b'] 

Sigma_0 =  ['z', 'd', 'x', 'a', 'w', 'y', 'c', 'b'] 

Sigma_1 =  ['xw', 'bc', 'ab', 'xz', 'bd', 'cd', 'yz', 'zw', 'xa', 'ad', 'yw', 'ac', 'xy'] 

Sigma_2 =  ['xzw', 'acd', 'yzw', 'abc', 'xyw', 'abd', 'xyz', 'bcd'] 

Betti numbers = [1, 0, 1, 0]
